In [5]:
import os
%pwd

'/Users/matt/Documents/deeplearning'

In [6]:
os.chdir('/Users/matt/Documents/deeplearning/')
%pwd

'/Users/matt/Documents/deeplearning'

In [12]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
  root_dir: Path
  trained_model_path: Path
  updated_base_model_path: Path
  training_data: Path
  validation_data: Path
  params_epochs:int
  params_batch_size:int
  params_is_augmentation:bool
  params_image_size:list
  
@dataclass(frozen=True)
class PrepareCallbacksConfig:
  root_dir: str
  tensorboard_root_log_dir: str
  checkpoint_model_filepath: str  

In [13]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [14]:

class ConfigurationManager:
  def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])
    
  def get_prepare_callbacks_config(self)->PrepareCallbacksConfig:
    
    config=self.config.prepare_callbacks
    checkpoint_model_dir=os.path.dirname(config.checkpoint_model_filepath)
    
    create_directories([
      checkpoint_model_dir,
      config.tensorboard_root_log_dir
    ])
    
    preare_callbacks_config=PrepareCallbacksConfig(
      root_dir=config.root_dir,
      checkpoint_model_filepath=config.checkpoint_model_filepath,
      tensorboard_root_log_dir=config.tensorboard_root_log_dir
    )
    
    return preare_callbacks_config  
    
    
  def get_training_config(self) ->TrainingConfig:
    traing=self.config.training
    prepare_base_model=self.config.prepare_base_model
    params=self.params
    training_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images","training")
    validation_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images","validation")
    
    create_directories([traing.root_dir])
    
    training_config=TrainingConfig(
      root_dir=Path(traing.root_dir),
      trained_model_path=Path(traing.trained_model_path),
      updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
      training_data=Path(training_data),
      validation_data=Path(validation_data),
      params_epochs=params.EPOCHS,
      params_batch_size=params.BATCH_SIZE,
      params_is_augmentation=params.AUGMENTATION,
      params_image_size=params.IMAGE_SIZE
      )    
    
    return training_config  
      

In [15]:
import time

In [16]:
class PrepareCallbacks:
  def __init__(self,config:PrepareCallbacksConfig):
    self.config = config
    
  @property
  def _create_tb_callbacks(self):
    timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
    tb_running_log_dir =os.path.join(
      self.config.tensorboard_root_log_dir,
      f"tb_logs_at_{timestamp}"      
    )
    return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
  
  @property
  def _create_checkpoint_callbacks(self):
    checkpoint_callbacks=tf.keras.callbacks.ModelCheckpoint(
      filepath=self.config.checkpoint_model_filepath,
      save_best_only=True)
    return checkpoint_callbacks
    
  def get_tb_ckpt_callbacks(self):
    return [
      self._create_tb_callbacks, 
      self._create_checkpoint_callbacks
      ]

In [17]:
import os
import zipfile
import tensorflow as tf
import urllib.request as request
import time


In [18]:
class Training:
  def __init__(self,config:TrainingConfig):
    self.config = config
    
  def get_base_model(self):
    self.model=tf.keras.models.load_model(
      self.config.updated_base_model_path
    )
    
  
    
  def train_valid_generator(self):   
# rescale: 重缩放因子。默认为 None。如果是 None 或 0，不进行缩放，否则将数据乘以所提供的值（在应用任何其他转换之前）。
# rescale的作用是对图片的每个像素值均乘上这个放缩因子，这个操作在所有其它变换操作之前执行，在一些模型当中，
# 直接输入原图的像素值可能会落入激活函数的“死亡区”，因此设置放缩因子为1/255，把像素值放缩到0和1之间有利于模型的收敛，避免神经元“死亡”。 

    datagenerator_kwargs = dict(
        rescale = 1./255,
        # validation_split=0.20  #  validation_split: 浮点数。Float. 保留用于验证的图像的比例（严格在0和1之间）。  
    )

    dataflow_kwargs = dict(
        target_size=self.config.params_image_size[:-1],
        batch_size=self.config.params_batch_size,
        interpolation="bilinear"
    )

    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
    )

    self.valid_generator = valid_datagenerator.flow_from_directory(
        directory=self.config.training_data,
        shuffle=False,
        **dataflow_kwargs
    )
    
    train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        **datagenerator_kwargs
    )   

    self.train_generator = train_datagenerator.flow_from_directory(
        directory=self.config.validation_data,
        shuffle=True,
        **dataflow_kwargs
    )  
    
  @staticmethod
  def save_model(path:Path, model:tf.keras.Model):
    model.save(path)
    
  def train(self,callback_list:list):
    self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
    self.validition_steps = self.valid_generator.samples // self.valid_generator.batch_size
    
    self.model.fit(
        self.train_generator,
        epochs = self.config.params_epochs,
        steps_per_epoch=self.steps_per_epoch,
        validation_data=self.valid_generator,
        validation_steps=self.validition_steps,
        callbacks=callback_list
        )
    
    self.save_model(self.config.trained_model_path,self.model)
  

In [22]:
%pwd

'/Users/matt/Documents/deeplearning'

In [23]:
try:
  config_manager=ConfigurationManager()
  
  prepare_callbacks_config = config_manager.get_prepare_callbacks_config()
  prepare_callbacks = PrepareCallbacks(prepare_callbacks_config)
  callback_list=prepare_callbacks.get_tb_ckpt_callbacks() 
  
  training_config =config_manager.get_training_config()
  training=Training(training_config)
  training.get_base_model()
  training.train_valid_generator()
  training.train(
    callback_list=callback_list
    )
   
  
  
except Exception as e:
  raise e

[2023-10-17 14:48:49,199: INFO: common: yaml file: config/config.yaml loaded successfully]


[2023-10-17 14:48:49,221: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-17 14:48:49,227: INFO: common: created directory at: artifacts]
[2023-10-17 14:48:49,239: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2023-10-17 14:48:49,247: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2023-10-17 14:48:49,287: INFO: common: created directory at: artifacts/training]
Found 292 images belonging to 2 classes.
Found 98 images belonging to 2 classes.
6/6 [==============================] - ETA: 0s - loss: 13.4716 - accuracy: 0.4634

/Users/matt/opt/anaconda3/envs/chicken/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6/6 [==============================] - 126s 24s/step - loss: 13.4716 - accuracy: 0.4634 - val_loss: 14.2839 - val_accuracy: 0.4931
